# Priority Digest LLM

**Priority Digest LLM** is a lightweight tool designed to reduce cognitive overload and help knowledge workers focus on what truly matters.

Modern work is filled with constant interruptions — emails, chats, notifications — and it’s easy to end the day feeling busy but not productive.

This tool analyzes two simple inputs:

1. **Your core job objective**  
   *(the real purpose of your role, not a generic job description)*

2. **A short selection of your daily messages**  
   *(emails, chat excerpts, notes — only what feels relevant)*

Using this information, the model:

- Extracts potential tasks  
- Evaluates which ones truly support your role’s objectives  
- Classifies them using the **Eisenhower Priority Matrix**  
- Highlights what to do now, plan, delegate, or drop  

The goal is not to tell you how to work, but to give you clarity, reduce noise, and help you prioritize with intention.

You remain the final decision-maker — this is a guide, not a replacement for professional judgment.


In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print('No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!')
elif not api_key.startswith('sk-proj-'):
    print('An API key was found, but it doesn\'t start sk-proj-; please check you\'re using the right key - see troubleshooting notebook')
elif api_key.strip() != api_key:
    print('An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook')
else:
    print('API key found and looks good so far!')

API key found and looks good so far!


In [23]:
system_prompt = """
You are an AI assistant called Priority Digest LLM.

You MUST produce the Eisenhower Matrix in four separate visual blocks.
NEVER attempt to render a single ASCII grid.
Markdown in Jupyter does NOT preserve alignment across blocks.

The required format is:

# 📊 Eisenhower Matrix

## 🔴 Q1 — DO NOW (Urgent + Important)
**Tasks (max 5):**  
- [ ] Task 1  
- [ ] Task 2  

---

## 🟠 Q2 — PLAN (Not Urgent + Important)
**Tasks (max 5):**  
- [ ] Task 3  
- [ ] Task 4  

---

## 🟢 Q3 — DELEGATE (Urgent + Not Important)
**Tasks (max 5):**

---

## ⚫ Q4 — DROP (Not Urgent + Not Important)
**Tasks (max 5):**

---

IMPORTANT RULES:
- Do NOT repeat the user's job objective or notes in the output.
- List at most 5 tasks per quadrant.
- No extra commentary before or after the quadrants.

Be concise, practical and realistic. Do not invent tasks that are not implied by the input.
"""


You may adjust the input below according to your needs.

In [24]:
job_objective = """
Ensure accurate financial and operational planning for a portfolio of infrastructure projects,
with a monthly revenue of $4MM and a target gross margin of 12%.

Key responsibilities:
- Build and maintain a rolling 3-month forecast (revenue, costs, and GM%).
- Detect and explain monthly cost and margin deviations vs. budget.
- Provide finance and operations with timely, decision-ready reports.
- Flag risks and opportunities affecting GM% and cash collection.
"""

highlights = """
- Meeting with operations: they requested an updated November forecast broken down by project
  and cost category before Thursday 4 PM.
- Email from finance: they need a clear explanation of October cost deviations > 5% vs. budget
  for three specific projects.
- Quick chat with logistics: minor delay in materials for two low-revenue projects, no direct
  impact on total margin this month.
- Short call with sales: they mentioned a potential upsell on an existing project that could
  add +$250k revenue in December if approved on time.
"""

pending = """
- Prepare weekly performance report (revenue vs. forecast, GM%, and key variances) for Monday.
- Answer finance with a structured deviation analysis (root causes + corrective actions) for
  the three projects with highest variance.
- Schedule and run a working session with operations to validate new cost assumptions used
  in the November and December forecasts.
- Update the forecast model with the latest purchase orders and approved change orders.
- Review aged receivables and identify projects at risk of delayed cash collection.
"""

concerns = """
- Missing some updated cost data for subcontractors in two major projects, which may distort
  the November GM% forecast.
- Too many minor, low-impact requests (small reports, ad-hoc tables, informal chats) that
  consume time but do not materially affect decisions or margin.
- Lack of clear ownership for some corrective actions proposed in previous deviation analyses.
- Risk that the upsell opportunity mentioned by sales is not evaluated in time and is lost.
"""

daily_notes = f"""
Highlights:
{highlights}

Pending:
{pending}

Concerns:
{concerns}
"""


In [25]:
openai = OpenAI()
def priority_digest(job_objective: str, daily_notes: str, model: str = "gpt-4.1-mini"):
    user_prompt = f"""
Here is the user's core job objective:
{job_objective}
Here are the user's daily notes (Highlight, Pending items, Concerns):
{daily_notes}
Please generate the Priority Digest following the instructions in the system prompt.
"""
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
    )
    return response.choices[0].message.content

In [27]:
digest = priority_digest(job_objective, daily_notes)
print(digest)

# 📊 Eisenhower Matrix

## 🔴 Q1 — DO NOW (Urgent + Important)
**Tasks (max 5):**  
- [ ] Provide finance with structured deviation analysis for the three projects with highest cost variance > 5%.  
- [ ] Deliver updated November forecast (broken down by project and cost category) to operations before Thursday 4 PM.  
- [ ] Prepare weekly performance report (revenue vs. forecast, GM%, key variances) for Monday.  
- [ ] Update forecast model with latest purchase orders and approved change orders.  
- [ ] Schedule and run working session with operations to validate new cost assumptions for November and December forecasts.  

---

## 🟠 Q2 — PLAN (Not Urgent + Important)
**Tasks (max 5):**  
- [ ] Review aged receivables and identify projects at risk of delayed cash collection.  
- [ ] Clarify ownership and accountability for corrective actions from previous deviation analyses.  
- [ ] Evaluate and plan integration of sales upsell opportunity (+$250k revenue) for December forecast.  
- [ ] D